In [ ]:
## Streamlined methods to prepare weight categories for a HAF weighted analysis, 
## meant to be used after running the sdd_clip function in R

#Author: Kaitlin Lubetkin
#Created: 9/24/21
#Last Edited: 2/15/22


In [7]:
## Prep workspace and define functions
import os
import arcpy

# Union all strata shapefiles created using the sdd_clip function in R
def union_strata(season):
    frame_list = []
    sample_frame_path = sample_frame_path_base + season + "_SUA"
    name_list = ["lmf1", "lmf2"] + ["s" + str(i) for i in range(1, maxS[season] + 1)]
    for file in os.listdir(sample_frame_path):
        if file.endswith(".shp"):
            f = file[:len(file)-4]
            frame_list.append(f)
            arcpy.management.MakeFeatureLayer(sample_frame_path + "\\" + file, f)
            with arcpy.da.SearchCursor(f, ["unique_id", "name"]) as cursor:
                for row in cursor:
                    thisID = row[0]
                    thisName = row[1]
                    del row
            del cursor
            arcpy.management.CalculateField(f, "id_" + thisID, '"' + thisID + '"')
            arcpy.management.CalculateField(f, "name_" + thisID, '"' + thisName + '"')
            arcpy.management.DeleteField(f, "unique_id")
            arcpy.management.DeleteField(f, "name")
    arcpy.analysis.Union(frame_list, analysis_gdb + "\\strata_" + season + "_UNION", "NO_FID")
    arcpy.management.RepairGeometry(analysis_gdb + "\\strata_" + season + "_UNION", "DELETE_NULL")
    for f in frame_list:
        arcpy.management.Delete(f)

    arcpy.management.AddField("strata_" + season + "_UNION", "area_hectares", "DOUBLE")
    arcpy.management.CalculateGeometryAttributes("strata_" + season + "_UNION", 
                                                 "area_hectares AREA", 
                                                 area_unit = "HECTARES")
    arcpy.management.AddField("strata_" + season + "_UNION", "strata_combo", "TEXT", field_length = 80)
    arcpy.management.AddField("strata_" + season + "_UNION", "Design_Stratum", "TEXT", field_length = 255)
    addr = ["!id_" + n + "!" for n in name_list]
    arcpy.management.CalculateField("strata_" + season + "_UNION", "strata_combo", 
                                    "ConcatAddr(" + ','.join([a for a in addr]) + ")", 
                                    "PYTHON3", 
                                    "def ConcatAddr(*args): return ''.join([str(i) for i in args if i not in(None,' ')]) ", 
                                    "TEXT", 
                                    "NO_ENFORCE_DOMAINS")
    arcpy.management.SelectLayerByAttribute("strata_" + season + "_UNION", "NEW_SELECTION", "area_hectares >= 1.5", None)
    addr = ["!name_" + n + "!" for n in name_list]
    arcpy.management.CalculateField("strata_" + season + "_UNION", "Design_Stratum", 
                                    "ConcatAddr(" + ','.join([a for a in addr]) + ")", 
                                    "PYTHON3", 
                                    "def ConcatAddr(*args): return ''.join([str(i) for i in args if i not in(None,' ')]) ", 
                                    "TEXT", 
                                    "NO_ENFORCE_DOMAINS")
    arcpy.management.SelectLayerByAttribute("strata_" + season + "_UNION", "CLEAR_SELECTION")
    for name in name_list:
        arcpy.management.DeleteField("strata_" + season + "_UNION", "id_" + name)
        arcpy.management.DeleteField("strata_" + season + "_UNION", "name_" + name)

        
# Prepare dissolved version and calculate weight categories
def dissolve_strata(season):
    #Dissolve based on the "strata_combo" & "Design_Stratum"
    arcpy.management.Dissolve("strata_" + season + "_UNION", 
                              analysis_gdb + "\\strata_" + season + "_UNION_Dissolve", 
                              "strata_combo;Design_Stratum")
    #Calculate hectares
    arcpy.management.CalculateGeometryAttributes("strata_" + season + "_UNION_Dissolve", 
                                                 "area_hectares_" + season + " AREA", 
                                                 area_unit = "HECTARES")
    #Add weight cat field
    arcpy.management.AddField("strata_" + season + "_UNION_Dissolve", "wgtcat_" + season, "SHORT")
    arcpy.management.CalculateField("strata_" + season + "_UNION_Dissolve", 
                                    "wgtcat_" + season, 
                                    "!OBJECTID!", "PYTHON3")

In [8]:
parent_folder = r'C:\Users\klubetkin\Desktop\__TRANSFER\_HAF site-scale analyses\WY\WindRiverFront_2022'
sample_frame_path_base = parent_folder + '\\2_Methods\\WindRiverFront_'
analysis_gdb = parent_folder + "\\1_Analysis\\" + 'WindRiverFront_RockSprings_SiteScale_Analysis_Inputs_20220308.gdb'
fine_scale = "WindRiverFront"

# The maximum TerrADat sample frame number (will be 2 less than the max sXX.shp if the 2 LMF strata are included)
maxS = {'spring': 14, 'summer': 14, 'winter': 14}


## Union all strata for each season
for thisSeason in ["summer", "winter"]:
    union_strata(thisSeason)
del thisSeason

In [ ]:
## Next, manually fix slivers (<1 ha) by giving them the strata_combo and Design_Stratum of the adjacent polygon

# also double check very small non-quite-slivers (1-1.5 ha) and give own name or split as 
# needed to merge with adjacent larger polygons

# Then proceed with next Jupyter cell

In [14]:
## Dissolve each season
#dissolve_strata("spring")
#dissolve_strata("summer")
dissolve_strata("winter")

In [ ]:
## Double check small (1-2 ha) polygons
## If necessary, return to the UNION and reassign strata combo & sample design, then redo the Dissolve

In [18]:
## Add weight cats from UNION_Dissolve to the SDD points   
#for thisSeason in ["_spring", "_summer", "_winter"]:
for thisSeason in ["_summer", "_winter"]:
    arcpy.management.AddField(fine_scale + r"_SDD_points\TerrestrialPointEvaluation", "wgtcat" + thisSeason, "SHORT")
    arcpy.analysis.SpatialJoin(fine_scale + r"_SDD_points\TerrestrialPointEvaluation", 
                               "strata" + thisSeason + "_UNION_Dissolve", 
                               analysis_gdb + "\\TerrPt_spjoin" + thisSeason)
    arcpy.management.AddJoin(fine_scale + r"_SDD_points\TerrestrialPointEvaluation", "TerrestrialVisitID", 
                             "TerrPt_spjoin" + thisSeason, "TerrestrialVisitID", "KEEP_ALL")
    arcpy.management.CalculateField(fine_scale + r"_SDD_points\TerrestrialPointEvaluation", 
                                    "TerrestrialPointEvaluation.wgtcat" + thisSeason, 
                                    "!TerrPt_spjoin" + thisSeason + ".wgtcat" + thisSeason + "_1!", "PYTHON3")
    arcpy.management.RemoveJoin(fine_scale + r"_SDD_points\TerrestrialPointEvaluation", "TerrPt_spjoin" + thisSeason)
    arcpy.management.Delete(analysis_gdb + "\\TerrPt_spjoin" + thisSeason)
    
## Add blank LMF-related fields to the SDD points
arcpy.management.AddField(fine_scale + r"_SDD_points\TerrestrialPointEvaluation", "segment_id", "TEXT", 20)
for thisSeason in ["_spring", "_summer", "_winter"]:
    arcpy.management.AddField(fine_scale + r"_SDD_points\TerrestrialPointEvaluation", "area_ratio" + thisSeason, "DOUBLE")     
del thisSeason
## If any SDD points fall in a LMF segment, will need to manually populate those fields
arcpy.management.SelectLayerByLocation(in_layer=fine_scale + r"_SDD_points\TerrestrialPointEvaluation", 
                                       overlap_type="INTERSECT", 
                                       select_features=fine_scale + r"_LMF_strata_segments\segment_" + fine_scale, 
                                       selection_type="NEW_SELECTION")
result = arcpy.management.GetCount(fine_scale + r"_SDD_points\TerrestrialPointEvaluation")
if int(result[0]) > 0:
    print("{} TerrestrialPointEvaluation plots fall in an LMF segment".format(result[0]))
    print("Must manually populate segment_id and area_ratio.")

In [19]:
## Project points from RATINGS gdb into Albers Equal Area and save in the analysis inputs gdb
base_name = "WindRiverFrontQC"
albers = 'PROJCS["NAD_1983_Albers",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-96.0],PARAMETER["Standard_Parallel_1",29.5],PARAMETER["Standard_Parallel_2",45.5],PARAMETER["Latitude_Of_Origin",23.0],UNIT["Meter",1.0]]'
arcpy.management.Project("RATINGS_gdb\\" + base_name + "_HAF", 
                         analysis_gdb + "\\" + fine_scale + "_HAF", 
                         albers)

# Run each season
for thisSeason in [["_S3_Spring", "_spring"], ["_S4_Summer", "_summer"], ["_S6_Winter", "_winter"]]:
#for thisSeason in [["_S6_Winter", "_winter"]]:
    arcpy.management.Project("RATINGS_gdb\\" + base_name + thisSeason[0], 
                             analysis_gdb + "\\" + fine_scale + thisSeason[0], 
                             albers)
    # Split out TerrADat points
    arcpy.management.SelectLayerByAttribute(fine_scale + thisSeason[0], "NEW_SELECTION", "Source='TerrADat'", None)
    arcpy.management.CopyFeatures(fine_scale + thisSeason[0], analysis_gdb + "\\tdat_" + fine_scale + thisSeason[1])
    # Split out LMF points
    arcpy.management.SelectLayerByAttribute(fine_scale + thisSeason[0], "NEW_SELECTION", "Source='LMF'", None)
    arcpy.management.CopyFeatures(fine_scale + thisSeason[0], analysis_gdb + "\\lmf_" + fine_scale + thisSeason[1])
    # Add fields for weight cat, segment, and area_ratio
    arcpy.management.AddField("tdat_" + fine_scale + thisSeason[1], "wgtcat" + thisSeason[1], "SHORT")
    arcpy.management.AddField("tdat_" + fine_scale + thisSeason[1], "segment_id", "TEXT", 20)
    arcpy.management.AddField("tdat_" + fine_scale + thisSeason[1], "area_ratio" + thisSeason[1], "DOUBLE")
    arcpy.management.AddField("lmf_" + fine_scale + thisSeason[1], "wgtcat" + thisSeason[1], "SHORT")
    arcpy.management.AddField("lmf_" + fine_scale + thisSeason[1], "segment_id", "TEXT", 20)
    arcpy.management.AddField("lmf_" + fine_scale + thisSeason[1], "area_ratio" + thisSeason[1], "DOUBLE")
    # Join TerrADat points to PointEvaluation fc and calculate additional fields
    arcpy.management.AddJoin("tdat_" + fine_scale + thisSeason[1], "PrimaryKey",
                             fine_scale + r"_SDD_points\TerrestrialPointEvaluation", "TerrADatPrimaryKey", 
                             "KEEP_ALL")
    arcpy.management.CalculateField("tdat_" + fine_scale + thisSeason[1], 
                                    "wgtcat" + thisSeason[1], 
                                    "!TerrestrialPointEvaluation.wgtcat" + thisSeason[1] + "!", "PYTHON3")
    arcpy.management.CalculateField("tdat_" + fine_scale + thisSeason[1], 
                                    "segment_id", 
                                    "!TerrestrialPointEvaluation.segment_id!", "PYTHON3")
    arcpy.management.CalculateField("tdat_" + fine_scale + thisSeason[1], 
                                    "area_ratio" + thisSeason[1], 
                                    "!TerrestrialPointEvaluation.area_ratio" + thisSeason[1] + "!", "PYTHON3")
    arcpy.management.RemoveJoin("tdat_" + fine_scale + thisSeason[1], "TerrestrialPointEvaluation")
    
    # Spatially join LMF points to dissolved seasonal strata and calculate weight cat
    arcpy.analysis.SpatialJoin("lmf_" + fine_scale + thisSeason[1], 
                               "strata" + thisSeason[1] + "_UNION_Dissolve", 
                               analysis_gdb + "\\LMFseg_spjoin" + thisSeason[1])
    arcpy.management.AddJoin("lmf_" + fine_scale + thisSeason[1], "PrimaryKey", 
                             "LMFseg_spjoin" + thisSeason[1], "PrimaryKey", "KEEP_ALL")
    arcpy.management.CalculateField("lmf_" + fine_scale + thisSeason[1], 
                                    "wgtcat" + thisSeason[1], 
                                    "!LMFseg_spjoin" + thisSeason[1] + ".wgtcat" + thisSeason[1] + "_1!", "PYTHON3")
    arcpy.management.RemoveJoin("lmf_" + fine_scale + thisSeason[1], "LMFseg_spjoin" + thisSeason[1])
    arcpy.management.Delete(analysis_gdb + "\\LMFseg_spjoin" + thisSeason[1])
    
    # Spatially join LMF points to seasonal segments and calculate segment_id & area_ratio
    arcpy.analysis.SpatialJoin("lmf_" + fine_scale + thisSeason[1], 
                               fine_scale + r"_LMF_strata_segments\segment_" + fine_scale + thisSeason[1], 
                               analysis_gdb + "\\LMFseg_spjoin" + thisSeason[1])
    arcpy.management.AddJoin("lmf_" + fine_scale + thisSeason[1], "PrimaryKey", 
                             "LMFseg_spjoin" + thisSeason[1], "PrimaryKey", "KEEP_ALL")
    arcpy.management.CalculateField("lmf_" + fine_scale + thisSeason[1], 
                                    "segment_id", 
                                    "!LMFseg_spjoin" + thisSeason[1] + ".segment_id_1!", "PYTHON3")
    arcpy.management.CalculateField("lmf_" + fine_scale + thisSeason[1], 
                                    "area_ratio" + thisSeason[1], 
                                    "!LMFseg_spjoin" + thisSeason[1] + ".area_ratio!", "PYTHON3")
    arcpy.management.RemoveJoin("lmf_" + fine_scale + thisSeason[1], "LMFseg_spjoin" + thisSeason[1])
    arcpy.management.Delete(analysis_gdb + "\\LMFseg_spjoin" + thisSeason[1])
    
print("Done!")

Done!


In [ ]:
## Double check that none of the seasonal tdat or lmf points has NULL for wgt cat
## Then, return to the seasonal rmd scripts

<Result 'true'>